In [4]:
from datetime import datetime, date, timedelta
from typing import Dict, List
import os
from flask import Flask
from flask_sqlalchemy import SQLAlchemy
from sqlalchemy.ext.declarative import declared_attr
import uuid
from sqlalchemy.dialects.postgresql import UUID


app = Flask(__name__)

app.config['SQLALCHEMY_DATABASE_URI'] = "sqlite:///"
app.config["SQLALCHEMY_TRACK_MODIFICATIONS"] = False

db = SQLAlchemy(app)


In [5]:
class Business(db.Model):  # type: ignore
    """ Business parent model """
    __tablename__ = "business"

    id = db.Column(db.Integer, primary_key=True)
    employees = db.relationship('User', backref="employer")

    
    
class User(db.Model):  # type: ignore
    """ User parent model """
    __tablename__ = "user"

    id = db.Column(db.Integer, primary_key=True)
    employer_id = db.Column(db.Integer, db.ForeignKey('business.id'))
    
   

In [6]:
db.create_all()

In [7]:
b1 = Business()
b2 = Business()
u1 = User(employer_id=1)
u2 = User(employer_id=1)
u3 = User(employer_id=2)

db.session.add(b1)
db.session.add(b2)
db.session.add(u1)
db.session.add(u2)
db.session.add(u3)

db.session.commit()

In [14]:
print('b1.employees: ', b1.employees)

print('b1.employees: ', b1.employees)
print('tye_ b2.employees: ', type(b2.employees))



b1.employees:  []
b1.employees:  []
tye_ b2.employees:  <class 'sqlalchemy.orm.collections.InstrumentedList'>


In [13]:
b1.employees = []
db.session.commit()

In [6]:
db.session.delete(b1)
db.session.commit()

In [10]:
Business.query.all()
User.query.all()

[<User 3>]

In [ ]:
class User(db.Model):  # type: ignore
    """ User parent model """
    __tablename__ = "user"

    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String)
    level = db.Column(db.Integer)
    
    def __repr__(self):
        return '<User: {} | Name: {} | Level: {}>'.format(self.id, self.name, self.level)

    
    def update(self, data_changes):
        for key, val in data_changes.items():
            setattr(self, key, val)
        return self
    
    def update_diverging(self, data_changes):
        for k, v in data_changes.items():
            if data_changes[k] is not None and data_changes[k] != getattr(self, k):
                setattr(self, k, v)
        return self
    

In [ ]:
u1 = User(name='Thomas', level=5)
u2 = User(name='Tobi', level=2)

db.session.add(u1)
db.session.add(u2)
db.session.commit()

data_changes = {
    'name': 'Egon'
}

print(u1)
print(u2)

In [ ]:
u1.update_diverging(data_changes)
u2.update_diverging(data_changes)

db.session.commit()

In [ ]:
print(u1)
print(u2)

In [2]:
class Business(db.Model):  # type: ignore
    """ Business parent model """
    __tablename__ = "business"

    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(24))
    created_on = db.Column(db.DateTime, default=datetime.utcnow)
    created_by = db.Column(db.Integer,  db.ForeignKey('user.id', name='fk_user_business_created_by'))
    employees = db.relationship('User', backref="employer", primaryjoin='User.employer_id==Business.id', cascade="all, delete-orphan")

    
    
class User(db.Model):  # type: ignore
    """ User parent model """
    __tablename__ = "user"

    id = db.Column(db.Integer, primary_key=True)
    employer_id = db.Column(db.Integer, db.ForeignKey('business.id', name='fk_business_user_employer_id'))
    created_businesses = db.relationship('Business', backref='creator', order_by="desc(Business.created_on)", primaryjoin='Business.created_by==User.id', post_update=True)
    items = db.relationship('Item', backref="user", lazy='select')

    
    
class Item(db.Model):  # type: ignore
    """ User parent model """
    __tablename__ = "item"

    id = db.Column(db.Integer, primary_key=True)
    user_id = db.Column(db.Integer, db.ForeignKey('user.id', name='fk_user_item_user_id'))
    
    

In [3]:
#db.reflect()
print("committing") #https://stackoverflow.com/questions/24289808/drop-all-freezes-in-flask-with-sqlalchemy
db.session.remove()

print("Dopping all")
db.drop_all()

print("Creating all")
db.create_all()

committing
Dopping all
Creating all


In [4]:
b1 = Business(name="b1")
b2 = Business(name="b2")

u1 = User()
u2 = User()

i1 = Item()
i2 = Item()

db.session.add(b1)
db.session.add(b2)
db.session.add(u1)
db.session.add(u2)
db.session.add(i1)
db.session.add(i2)

db.session.commit()

b1.employees.append(u1)
b1.employees.append(u2)
u1.created_businesses.append(b1)
u2.created_businesses.append(b2)
u1.items.append(i1)
u1.items.append(i2)
db.session.commit()

print(b1.employees)
print(u1.employer.name)
print(u1.created_businesses)
print(b2.created_by)
print(u1.items)


[<User 1>, <User 2>]
b1
[<Business 1>]
2
[<Item 1>, <Item 2>]


In [ ]:
db.session.delete(b1)
db.session.commit()

print("All Businesses:", Business.query.all())
print("All Employees:", User.query.all())
print("All Item:", Item.query.all())


In [6]:
len_items = Business.query.filter_by(id = 1).options(subqueryload(User.items)).all()

NameError: name 'subqueryload' is not defined

In [10]:
b1 = Business.query.filter_by(id=1).options(subqueryload(User.items)).all()

ArgumentError: Mapped attribute "User.items" does not apply to any of the root entities in this query, e.g. mapped class Business->business. Please specify the full path from one of the root entities to the target attribute. 

In [9]:
from sqlalchemy.orm import subqueryload

In [ ]:
q = db.session.query(Tag.text) # @new:modified
q = q.join(Tag.article)
q = q.join(Article.project)
q = q.filter(Project.id == project_id)
q = q.filter(Article.date_created.between(now-timedelta(hours=1), now))
tags = q.all()
return tags # @not

In [13]:
b1_items = Item.query.join(Item.user).join(User.employer).filter(Business.id == 1).all()

In [14]:
b1_items

[<Item 1>, <Item 2>]

In [2]:
class Transaction(db.Model):  # type: ignore
    """ User parent model """
    __tablename__ = "transaction"

    code = db.Column(db.String(8), primary_key=True)
    item_tax_rate_type_code = db.Column(db.String(8), db.ForeignKey('tax_rate_type.code'), nullable=False)
    shipment_tax_rate_type_code = db.Column(db.String(8), db.ForeignKey('tax_rate_type.code'))
    gift_wrap_tax_rate_type_code = db.Column(db.String(8), db.ForeignKey('tax_rate_type.code'))
    

class TaxRateType(db.Model):  # type: ignore
    """ User parent model """
    __tablename__ = "tax_rate_type"

    code = db.Column(db.String(8), primary_key=True)
    transaction_items = db.relationship('Transaction', backref='item_tax_rate_type', foreign_keys='Transaction.item_tax_rate_type_code', lazy=True)
    transaction_shipments = db.relationship('Transaction', backref='shipment_tax_rate_type', foreign_keys='Transaction.shipment_tax_rate_type_code', lazy=True)
    transaction_gift_wraps = db.relationship('Transaction', backref='gift_wrap_tax_rate_type', foreign_keys='Transaction.gift_wrap_tax_rate_type_code', lazy=True)


    


In [3]:
db.create_all()

In [4]:
v1 = TaxRateType(code="A")
v2 = TaxRateType(code="B")
db.session.add(v1)
db.session.add(v2)

t1 = Transaction(code="T1", item_tax_rate_type_code="A", shipment_tax_rate_type_code="B", gift_wrap_tax_rate_type_code="A")
t2 = Transaction(code="T2", item_tax_rate_type_code="B", shipment_tax_rate_type_code="B", gift_wrap_tax_rate_type_code="B")

db.session.add(t1)
db.session.add(t2)

db.session.commit()



In [12]:
v2.transaction_shipments

[<Transaction T1>, <Transaction T2>]